Indicate which posts in Posts REDCap contain keywords in term dictionary.

In [ ]:
from constants import posts_token, all_posts
from functions import pull_redcap_report, preprocess_terms, remove_stopwords, preprocess_text, search_terms_in_text, redcap_upload
import json

# Pull all posts from REDCap
df = pull_redcap_report(posts_token, all_posts)

# Open Term Dictionary
with open("../term_dictionary/term_dictionary.json", "r") as f:
    term_dictionary = json.load(f)


term_dictionary = preprocess_terms(term_dictionary, stem = False)
preprocessed_term_dictionary = preprocess_terms(term_dictionary, stem = True)

# Lowercase df['text'] column
df['text'] = df['text'].astype(str).str.lower()

# Copy the text column for processing
df['text_processed'] = df['text'] 

# Apply the preprocessing functions
df = remove_stopwords(df, 'text_processed')
df = preprocess_text(df, 'text_processed')
df = search_terms_in_text(df, term_dictionary, preprocessed_term_dictionary)  

df = df[['record_id', 'keywords', 'keyword_categories']]
df = df[~((df['keywords'].isna() | (df['keywords'] == '')) &
          (df['keyword_categories'].isna() | (df['keyword_categories'] == '')))]
df['contains_keyword'] = '1'

print(len(df))
response = redcap_upload(df)

22881
